In [39]:
from keras import layers
from keras.models import Model
from mltu.tensorflow.model_utils import residual_block
from mltu.tensorflow.dataProvider import DataProvider
from mltu.preprocessors import ImageReader
from configs import ModelConfigs
from mltu.tensorflow.callbacks import TrainLogger
from mltu.tensorflow.metrics import CWERMetric
from tensorflow.keras.models import load_model
from mltu.annotations.images import CVImage
from mltu.transformers import ImageResizer, LabelIndexer, LabelPadding
from mltu.tensorflow.losses import CTCloss
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tqdm import tqdm
import tensorflow as tf
import pickle
import os

In [12]:
def create_model(input_dim, output_dim, activation="leaky_relu", dropout=0.2):
    inputs = layers.Input(shape=input_dim, name='input')
    input = layers.Lambda(lambda x: x / 255)(inputs)

    x1 = residual_block(input, 16, activation=activation, skip_conv=True, strides=1, dropout=dropout)
    x2 = residual_block(x1, 16, activation=activation, skip_conv=True, strides=2, dropout=dropout)
    x3 = residual_block(x2, 16, activation=activation, skip_conv=False, strides=1, dropout=dropout)
    x4 = residual_block(x3, 32, activation=activation, skip_conv=True, strides=2, dropout=dropout)
    x5 = residual_block(x4, 32, activation=activation, skip_conv=False, strides=1, dropout=dropout)
    x6 = residual_block(x5, 64, activation=activation, skip_conv=True, strides=1, dropout=dropout)
    x7 = residual_block(x6, 64, activation=activation, skip_conv=False, strides=1, dropout=dropout)

    squeezed = layers.Reshape((x7.shape[-3] * x7.shape[-2], x7.shape[-1]))(x7)
    bi_LSTM = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(squeezed)
    output = layers.Dense(output_dim + 1, activation="softmax", name="output")(bi_LSTM)
    model = Model(inputs=inputs, outputs=output)

    return model

In [13]:
configs = ModelConfigs()

In [20]:
train_images_path = "./dataset/IDCAR2015_Dataset/train/training_images/"
train_annotations_path = "./dataset/IDCAR2015_Dataset/train/training_localization_transcription"
val_images_path = "./dataset/IDCAR2015_Dataset/val/val_images/"
val_annotations_path = "./dataset/IDCAR2015_Dataset/val/val_localization_transcription"

In [21]:
def read_annotation_file(image_folder, annotations_folder):
    dataset, vocab, max_len = [], set(), 0

    # Get sorted lists of image and annotation files to ensure they match
    image_files = sorted(os.listdir(image_folder))
    annotation_files = sorted(os.listdir(annotations_folder))

    # Use zip to combine image and annotation files
    for image_file, annotation_file in tqdm(zip(image_files, annotation_files)):
        image_path = os.path.join(image_folder, image_file)
        annotation_path = os.path.join(annotations_folder, annotation_file)

        # Read the annotation file
        with open(annotation_path, 'r') as f:
            for line in f.readlines():
                line = line.strip().split()
                
                # Assuming the annotation contains the image name and the label
                label = line[1] if len(line) > 1 else "UNKNOWN"  # Handle missing label

                # Append image path and label to the dataset
                dataset.append([image_path, label])
                vocab.update(list(label))
                max_len = max(max_len, len(label))

    return dataset, sorted(vocab), max_len


In [22]:
train_dataset, train_vocab, max_train_len = read_annotation_file(train_images_path, train_annotations_path)
val_dataset, val_vocab, max_val_len = read_annotation_file(val_images_path, val_annotations_path)

1000it [00:00, 4800.00it/s]
500it [00:00, 4104.47it/s]


In [23]:
# Create training data provider
train_data_provider = DataProvider(
    dataset=train_dataset,
    skip_validation=True,
    batch_size=configs.batch_size,
    data_preprocessors=[ImageReader(CVImage)],
    transformers=[
        ImageResizer(configs.width, configs.height),
        LabelIndexer(configs.vocab),
        LabelPadding(max_word_length=configs.max_text_length, padding_value=len(configs.vocab))
    ],
)

# Create validation data provider
val_data_provider = DataProvider(
    dataset=val_dataset,
    skip_validation=True,
    batch_size=configs.batch_size,
    data_preprocessors=[ImageReader(CVImage)],
    transformers=[
        ImageResizer(configs.width, configs.height),
        LabelIndexer(configs.vocab),
        LabelPadding(max_word_length=configs.max_text_length, padding_value=len(configs.vocab))
    ],
)

In [24]:
model = create_model(input_dim=(configs.height, configs.width, 3), output_dim=len(configs.vocab))
padding_token = len(configs.vocab)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=configs.learning_rate), loss=CTCloss(), run_eagerly=False)
#model.summary()"

c:\Users\1288843.TRANSIT.000\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [30]:
# Define callbacks
earlystopper = EarlyStopping(monitor="val_CER", patience=10, mode='min', verbose=1)
checkpoint = ModelCheckpoint(f"{configs.model_path}/model.keras", monitor="val_CER", verbose=1, save_best_only=True, mode="min")
trainLogger = TrainLogger(configs.model_path)
tb_callback = TensorBoard(f"{configs.model_path}/logs", update_freq=1)
reduceLROnPlat = ReduceLROnPlateau(monitor="val_CER", factor=0.9, min_delta=1e-10, patience=5, verbose=1, mode="auto")


In [31]:
history = model.fit(
    train_data_provider,
    validation_data=val_data_provider,
    epochs=configs.train_epochs,
    callbacks=[earlystopper, checkpoint, trainLogger, reduceLROnPlat, tb_callback],
)

Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 140s 9s/step - loss: 777.5939 - val_loss: 862.8233 - learning_rate: 1.0000e-04
Epoch 2/100


c:\Users\1288843.TRANSIT.000\AppData\Local\anaconda3\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_CER` which is not available. Available metrics are: loss,val_loss
  current = self.get_monitor_value(logs)
c:\Users\1288843.TRANSIT.000\AppData\Local\anaconda3\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_CER available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
c:\Users\1288843.TRANSIT.000\AppData\Local\anaconda3\Lib\site-packages\keras\src\callbacks\callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_CER` which is not available. Available metrics are: loss,val_loss,learning_rate.
  callback.on_epoch_end(epoch, logs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 126s 7s/step - loss: 628.6485 - val_loss: 747.6307 - learning_rate: 1.0000e-04
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 133s 8s/step - loss: 484.5318 - val_loss: 601.3226 - learning_rate: 1.0000e-04
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 132s 8s/step - loss: 354.3723 - val_loss: 454.9121 - learning_rate: 1.0000e-04
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 128s 7s/step - loss: 248.7078 - val_loss: 310.7771 - learning_rate: 1.0000e-04
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 126s 7s/step - loss: 171.4536 - val_loss: 183.0749 - learning_rate: 1.0000e-04
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 128s 8s/step - loss: 121.6573 - val_loss: 108.1963 - learning_rate: 1.0000e-04
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 133s 8s/step - loss: 91.4814 - val_loss: 76.0702 - learning_rate: 1.0000e-04
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 146s 8s/step - loss: 73.7003 - val_loss: 61.4241 - learning_rate: 1.0000e-04
Epoch 10/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 163s 10s/step - loss: 63.2931 - 

In [32]:
model.save('textocr_model.h5')

In [37]:
def ctc_loss(y_true, y_red):
    return K.ctc_batch_cost(y_true, y_red, input_length, label_length)

In [38]:
model = load_model('textocr_model.h5', custom_objects={'CTCloss': ctc_loss})